## **TurboBatch User Dashboard**

### **Environment Set-up**
Before submitting your job please ensure you have created a new Google Cloud Project with an associated billing account and have the gcloud CLI working in your environment. Additionally, this dashboard requires certain packages be installed. Please make sure to `pip install -r requirements.txt` in your environment before continuing.

The scripts below will assist with any further steps:

**Ensure our system's service account has all required permissions to your project**

To do this, you can simply run our privileges.sh bash script in the terminal. Before running the command, make sure to first navigate to the helpers directory. Please type the following command, replacing [project-id] with your actual project's ID. Simple select 'yes' when prompted and complete the authorization process on the browser.

```./privileges.sh [project-id]```

### **Upload Input DataFrame into BigQuery Table**

Our system requires an input in the form of a BigQuery Table and writes to an output table in the same form. To streamline the process of the creation of these tables, simply run the cell below after changing the variables to upload and create an input table along with creating an empty output table in your project.

Please note that the input table must be formated with two columns: `row` (integers) and `prompt_and_text` (strings). See `example.csv` for reference.



In [2]:
from helpers.df import CSV_to_BigQuery

#TODO Change the below variables before running this cell
input_CSV_path = "example.csv"
project_id = "sampleproject-440900"
dataset_id = "test"
input_table_id = "input"
output_table_id = "output"

CSV_to_BigQuery(input_CSV_path, project_id, dataset_id, input_table_id, output_table_id)

Forbidden: 403 GET https://bigquery.googleapis.com/bigquery/v2/projects/sampleproject-440900/datasets/test?prettyPrint=false: Access Denied: Dataset sampleproject-440900:test: Permission bigquery.datasets.get denied on dataset sampleproject-440900:test (or it may not exist).

### **Submit Job**

Fill in the relevant information below and then run the cell to submit a job. Upon submitting a job, there will be an update every three seconds on the status of your job until completion.

Note:
- Currently supported model: "gpt-3.5-turbo"
- Leaving API_key empty ("") will result in "sample_response" being returned for all input data.

In [3]:
from helpers.jobSubmit import submitjob

#TODO Change the below variables before running this cell
username = "test_user"
password = "test_password"
Client_ID = "test"
User_Project_ID = "sampleproject-440900"
User_Dataset_ID = "test"
Input_Table_ID = "input"
Output_Table_ID = "output"
Model = "gpt-3.5-turbo"
API_key = ""

job, key = submitjob(username, password, Client_ID, User_Project_ID, User_Dataset_ID, Input_Table_ID, Output_Table_ID, Model, API_key)

2024-12-05 10:28:48,295 - INFO - 🔑 Retrieving API key...
2024-12-05 10:28:48,396 - INFO - API key retrieved successfully: 7b57da27-c470-434d-87a6-5f3320a0f410
2024-12-05 10:28:48,396 - INFO - 📋 Preparing job data...
2024-12-05 10:28:48,396 - INFO - 📤 Submitting job...
2024-12-05 10:28:48,559 - INFO - Job submitted successfully.
2024-12-05 10:28:48,569 - INFO - ⏳ Monitoring job status...
2024-12-05 10:28:48,570 - INFO - 🔎 Checking for Job Status...
2024-12-05 10:28:48,754 - ERROR - Failed to retrieve job status: 500, {"details":"'NoneType' object has no attribute 'get'","error":"Failed to retrieve job status"}

2024-12-05 10:28:48,754 - ERROR - ❌ Could not retrieve job status. Retrying...
2024-12-05 10:28:51,758 - INFO - 🔎 Checking for Job Status...
2024-12-05 10:28:51,880 - ERROR - Failed to retrieve job status: 500, {"details":"'NoneType' object has no attribute 'get'","error":"Failed to retrieve job status"}

2024-12-05 10:28:51,881 - ERROR - ❌ Could not retrieve job status. Retrying

### **Manually Check Job Status by Running Cell Below**

If you would like to check on your job status please run the cell below to do so. Note that this can only be run after the previous cell has been run. Make sure to wait a few seconds before trying to obtain the status. 

In [4]:
from helpers.jobStatus import check_job_status

check_job_status(job, Client_ID, key)

2024-12-05 10:29:16,250 - INFO - 🔎 Checking for Job Status...
2024-12-05 10:29:16,370 - INFO - Job 27173a7e-0abb-487a-b5ed-2a9eafc3e823 is complete. Processed 3 rows.


{'status': 'complete',
 'details': {'Client_ID': 'test',
  'Job_ID': '27173a7e-0abb-487a-b5ed-2a9eafc3e823',
  'current_row': 3,
  'total_rows': 3}}

### **Save Output BigQuery Table as a CSV**

If you prefer to have the output responses and corresponding rows saved in your local machine as a CSV rather than checking the BigQuery table in your project, run the cell below to access the output table.

In [ ]:
from helpers.df import BigQuery_to_CSV

#TODO Change the below variables before running this cell
project_id = "sampleproject-440900"
dataset_id = "test"
output_table_id = "output"
output_csv_path = "output.csv"

BigQuery_to_CSV(project_id, dataset_id, output_table_id, output_csv_path)